In [ ]:
from pyspark.sql import DataFrame
from pyspark import Row
from pyspark.sql.types import *
from itertools import chain
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext
import pyspark.sql.functions as F

In [ ]:
%run ../../SSB_Spark_tools/utils/listcode_functions.py

In [ ]:
%run ../../SSB_Spark_tools/data_processing/Utils.py

In [ ]:
tverrsnitt = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding')
test_dict = pakkut_parq(tverrsnitt, rootdf=True, rootvar=['personidentifikator'])

test_dict['loennOgTilsvarendeYtelser_koblet'] = test_dict['loennOgTilsvarendeYtelser_samledeYtelserFraArbeidsgiverPerBehandlingsart']\
            .join(test_dict['loennOgTilsvarendeYtelser'].select('loennOgTilsvarendeYtelser_id', 'overfoertFraBarn'),\
            'loennOgTilsvarendeYtelser_id', how='left')

In [ ]:
ba_kodeliste_df = spark.read.path('/kilde/ske/skatt/person/2019/kodelister/behandlingsart')

#Gjør om kodeliste til python liste
ba_kodeliste = []
for row in ba_kodeliste_df.rdd.collect():
    ba_kodeliste.append(row['kode_behandlingsart'])
    
negtest = ['MAT_OVERTID', 'TFRI_KOST_TJENESTE']
sjekk, sjekkdf = listcode_check(test_dict['loennOgTilsvarendeYtelser_koblet'], 'behandlingsart', negtest)
#sjekk, sjekkdf = kodeliste_sjekk(so_dict['loennOgTilsvarendeYtelser_koblet'], 'behandlingsart', ba_kodeliste)
if (sjekk == False):
    ikke_i_kodeliste = sjekkdf.filter(F.col('i_kodeliste')==False)
    ikke_i_kodeliste.show()

In [ ]:
test = listcode_lookup(test_dict['loennOgTilsvarendeYtelser_koblet'], 'behandlingsart', ba_kodeliste_df, ['kode_behandlingsart', 'kodetekst_behandlingsart'])    
test.toPandas()